This notebook shows how you can use different LLMs with Eunomia

In [1]:
import eunomia
from eunomia.agents import Eunomia
from eunomia.tools import EunomiaTools

In [2]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings

# Disable all warning messages
warnings.filterwarnings("ignore")

## Obtaining Water Stability Information from a MOF Paper

In [3]:
paper_id = 69
paper_file =f'../data/{paper_id}.pdf'
docs_processor = eunomia.LoadDoc(file_name=paper_file, encoding="utf8")
sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments', 'references\n'],
                                             chunk_size=1000, chunk_overlap=20, chunking_type='fixed-size')

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

Embedding_model = 'text-embedding-ada-002' 
faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))

## Using OPENAI's GPT-4o model for the agent

In [5]:
tools = EunomiaTools(tool_names = ['read_doc', 'eval_justification', 'recheck_justification'], vectorstore=faiss_index).get_tools()
agent = Eunomia(tools=tools, model='gpt-4o', get_cost=True)
agent.run(prompt="Read the document and find the MOFS and their water stability.\
            \Check your answers and if the reasoning is not valid, try to find better reasons.")



> Entering new AgentExecutor chain...
I need to read the document to find information about the water stability of metal-organic frameworks (MOFs) and their justifications. 

Action: read_doc
Action Input: Read the document and find the MOFS and their water stability.Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 9133 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

k=9 results hitting the token limit for the chosen LLM. Reducing k and retrying...

Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8458 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

k=8 results hitting the token limit for the chosen LLM. Reducing k and retrying...


Observati

Thought:I have rechecked the justifications for the water stability of the MOFs mentioned in the document. Here are the final validated details:

1. **MOF: Zr /C0O**
   - **Water Stability:** Stable
   - **Probability Score:** 0.9
   - **Justification:** 
     - "Thermogravimetric analysis (TGA) and IR spectroscopy were therefore used to qualitatively assess the hydrophobic character of these solids. TGA measurements have shown that the samples exhibit almost no loss of free water."
     - "Optical water vapor adsorption isotherms have been collected (Figure S29) and show that these solids can be considered as being more hydrophobic than the UiO-66 counterpart which is fully loaded with water at intermediate humidity rate ( P/P 0=0.5):[11]Only a little adsorption occurs at low pressure, particularly for the smaller pore size MIL-140A and B solids, probably corresponding to coordination to the metal sites; at higher pressure, a filling of the pores starts, as typically observed for some

'1. **Zr /C0O**: Stable, Probability Score: 0.9\n2. **UiO-66**: Stable, Probability Score: 1\n3. **MIL-140**: Stable, Probability Score: 1\n4. **ZrO(O2C-R-CO2)**: Stable, Probability Score: 1\n5. **[Zr6O4(OH)4(O2C-R-CO2)6]**: Stable, Probability Score: 1'

## Using Anthropic's Sonnet 3.5 model for the agent

In [8]:
#!pip install langchain-anthropic==0.1.1
import os
from langchain_anthropic import ChatAnthropic
os.environ['ANTHROPIC_API_KEY'] = YOUR_ANTHROPIC_API_KEY


model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0.3, max_tokens=8192)
tools = EunomiaTools(tool_names = ['read_doc', 'eval_justification', 'recheck_justification'], vectorstore=faiss_index).get_tools()
agent = Eunomia(tools=tools, model=model, get_cost=True)
agent.run(prompt="Read the document and find the MOFS and their water stability.\
            \Check your answers and if the reasoning is not valid, try to find better reasons.")



> Entering new AgentExecutor chain...
To answer this question, I need to first read the document to find information about MOFs and their water stability. Then, I'll need to evaluate the justifications for each MOF's water stability and recheck if necessary.

Action: read_doc
Action Input: Read the document and find the MOFS and their water stability. Check your answers and if the reasoning is not valid, try to find better reasons.
Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 9133 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

k=9 results hitting the token limit for the chosen LLM. Reducing k and retrying...

Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens. However, your messages resulted in 8458 tokens. Please reduce the length of the messages.", 'type': 'invalid_

'After reading the document and evaluating the justifications, here are the MOFs and their water stability:\n\n1. Zr /C0O: Unstable\nJustification: It has an inherently low hydrolytic stability, undergoing hydrolysis of the Zr /C0O bonds when exposed to water.\n\n2. UiO-66: Stable\nJustification: It remains unaltered upon water adsorption/desorption cycles, switching reversibly between dehydroxylated and hydroxylated versions.\n\n3. MIL-140: Stable\nJustification: Based on XRPD analysis, MIL-140 compounds are hydrothermally stable. They show no loss of crystallinity after overnight dispersion in deionized water at 100°C, and no peaks from recrystallized dicarboxylate linkers are observed after hydrothermal treatment.\n\nFor the following MOFs, no specific information about their water stability was provided in the document:\n4. MIL-140A\n5. MIL-140B\n6. MIL-140C\n7. MIL-140D\n\nThe reasoning for the water stability of Zr /C0O, UiO-66, and MIL-140 has been validated and is considered ac